In [104]:
!pip install yfinance
!pip install plotly


[notice] A new release of pip is available: 25.2 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.2 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Question 1 - Extracting Tesla Stock Data Using yfinance

In [105]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.io as pio
pio.renderers.default = "iframe"
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [106]:
tesla = yf.Ticker("TSLA")

tesla_data = tesla.history(period="max")

tesla_data.reset_index(inplace=True)

tesla_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2010-06-29 00:00:00-04:00,1.266667,1.666667,1.169333,1.592667,281494500,0.0,0.0
1,2010-06-30 00:00:00-04:00,1.719333,2.028000,1.553333,1.588667,257806500,0.0,0.0
2,2010-07-01 00:00:00-04:00,1.666667,1.728000,1.351333,1.464000,123282000,0.0,0.0
3,2010-07-02 00:00:00-04:00,1.533333,1.540000,1.247333,1.280000,77097000,0.0,0.0
4,2010-07-06 00:00:00-04:00,1.333333,1.333333,1.055333,1.074000,103003500,0.0,0.0


## Question 2 - Extracting Tesla Revenue Data Using Webscraping

In [107]:
url = "https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue"

headers = {
    "User-Agent": "Mozilla/5.0"
}

html_data = requests.get(url, headers=headers).text

soup = BeautifulSoup(html_data, "html.parser")


In [108]:
tesla_revenue = pd.DataFrame(columns=["Date", "Revenue"])

tables = soup.find_all("table")

for table in tables:
    if "Tesla Quarterly Revenue" in str(table):

        for row in table.find_all("tr")[1:]:
            cols = row.find_all("td")

            if len(cols) != 0:
                tesla_revenue = pd.concat(
                    [
                        tesla_revenue,
                        pd.DataFrame(
                            {
                                "Date": [cols[0].text],
                                "Revenue": [cols[1].text],
                            }
                        ),
                    ],
                    ignore_index=True,
                )

In [109]:
tesla_revenue["Revenue"] = (
    tesla_revenue["Revenue"]
    .str.replace(",", "")
    .str.replace("$", "")
)

tesla_revenue = tesla_revenue[tesla_revenue["Revenue"] != ""]
tesla_revenue.dropna(inplace=True)

tesla_revenue["Revenue"] = tesla_revenue["Revenue"].astype(float)

tesla_revenue.tail()

,Date,Revenue
55,2012-03-31,30.0
56,2011-12-31,39.0
57,2011-09-30,58.0
58,2011-06-30,58.0
59,2011-03-31,49.0


## Question 3 - Extracting GameStop Stock Data Using yfinance

In [110]:
gme = yf.Ticker("GME")

gme_data = gme.history(period="max")

gme_data.reset_index(inplace=True)

gme_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2002-02-13 00:00:00-05:00,1.620128,1.693350,1.603296,1.691666,76216000,0.0,0.0
1,2002-02-14 00:00:00-05:00,1.712707,1.716074,1.670626,1.683250,11021600,0.0,0.0
2,2002-02-15 00:00:00-05:00,1.683250,1.687458,1.658001,1.674834,8389600,0.0,0.0
3,2002-02-19 00:00:00-05:00,1.666418,1.666418,1.578047,1.607504,7410400,0.0,0.0
4,2002-02-20 00:00:00-05:00,1.615920,1.662210,1.603296,1.662210,6892800,0.0,0.0


## Question 4 - Extracting GameStop Revenue Data Using Webscraping

In [111]:
url = "https://www.macrotrends.net/stocks/charts/GME/gamestop/revenue"

headers = {
    "User-Agent": "Mozilla/5.0"
}

html_data = requests.get(url, headers=headers).text

soup = BeautifulSoup(html_data, "html.parser")

In [112]:
gme_revenue = pd.DataFrame(columns=["Date", "Revenue"])

tables = soup.find_all("table")

for table in tables:
    if "GameStop Quarterly Revenue" in str(table):

        for row in table.find_all("tr")[1:]:
            cols = row.find_all("td")

            if len(cols) != 0:
                gme_revenue = pd.concat(
                    [
                        gme_revenue,
                        pd.DataFrame(
                            {
                                "Date": [cols[0].text],
                                "Revenue": [cols[1].text],
                            }
                        ),
                    ],
                    ignore_index=True,
                )

In [113]:
gme_revenue["Revenue"] = (
    gme_revenue["Revenue"]
    .str.replace(",", "")
    .str.replace("$", "")
)

gme_revenue = gme_revenue[gme_revenue["Revenue"] != ""]
gme_revenue.dropna(inplace=True)

gme_revenue["Revenue"] = gme_revenue["Revenue"].astype(float)

gme_revenue.tail()

,Date,Revenue
54,2012-04-30,2002.0
55,2012-01-31,3579.0
56,2011-10-31,1947.0
57,2011-07-31,1744.0
58,2011-04-30,2281.0


## Question 5 - Tesla Stock and Revenue Dashboard

In [114]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import pandas as pd

def make_graph(stock_data, revenue_data, stock):

    stock_data["Date"] = pd.to_datetime(stock_data["Date"])
    revenue_data["Date"] = pd.to_datetime(revenue_data["Date"])

    fig = make_subplots(
        rows=2, cols=1,
        shared_xaxes=True,
        subplot_titles=("Stock Price", "Revenue"),
        vertical_spacing=0.3
    )

    fig.add_trace(
        go.Scatter(
            x=stock_data["Date"],
            y=stock_data["Close"],
            name="Stock Price"
        ),
        row=1, col=1
    )

    fig.add_trace(
        go.Scatter(
            x=revenue_data["Date"],
            y=revenue_data["Revenue"],
            name="Revenue"
        ),
        row=2, col=1
    )

    fig.update_layout(
        title_text=stock + " Stock and Revenue Dashboard",
        height=900,
        showlegend=False
    )

    fig.show()


In [115]:
make_graph(tesla_data, tesla_revenue, "Tesla")

## Question 6 - GameStop Stock and Revenue Dashboard

In [116]:
make_graph(gme_data, gme_revenue, "GameStop")